# Wrangle and Analyze Data of a Twitter Account


## Table of Contents
- [1. Introduction](#intro)
- [2. Gather data](#gather)
- [3. Assess data](#assess)
- [4. Clean data](#clean)
- [5. Store](#store)


<a id='gather'></a>
## 1. Introduction

This project is an analysis of correlation between the Covid-19 cases and the political environment of different countries. Goal is to find answers or at least indicators to questions like: 
- Did the countries which had more success in containing the amount of Covid-19 cases something in common? 
- Is there a correlation in  Gross domestic product, Human Development Index or political ideology with the amount of Covid-19 cases of the country.

Main goal of this project is to generate a comprehensive exploratory and explanatory data analysis of the gathered data. The data analysis process is distributed over three ipynb-files: gather_clean_Covid19.ipynb, exploration_Covid19.ipynb and slide_deck_Covid19.ipynb.

Firstly, as part of gather_clean_Covid19.ipynb data is gathered from different sources: The Covid-19 data of this project is retrieved via programmatically downloaded csv-files from the GitHub repository [Covid-19](https://github.com/CSSEGISandData/COVID-19) and additional data about countries is retrieved via the wikipedia API. Secondly, the data from the different sources is visually and programmatically assessed to be cleaned.
The exploratory and explanatory data analysis of the gathered data is performed in exploration_Covid19.ipynb. Finally the findings are presented in slide_deck_Covid19.ipynb.

In [1435]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import date
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import os # to work with local directory
import re
import wptools
import json # to create json file from python dictionary
import time # for timer 
sns.set()

<a id='intro'></a>
## 2. Gather data

####  Data is gathered from three different sources of data as described in steps below:

1. Fatality, confirmed cases, recovered cases and data by country is retrieved via programmatically downloaded csv-files from the GitHub repository [Covid-19](https://github.com/CSSEGISandData/COVID-19).
2. Additional data is retrieved via the wptools API from different wikipedia articles.

### a. Read data from programmatically download csv-file

In [1436]:
# Gather data from John Hopkins GitHub 
df_JHU_Fatality = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_JHU_Confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_JHU_Recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
df_JHU_Countries = pd.read_csv('https://raw.githubusercontent.com/RRighart/covid-19/master/countries.csv')

In [1437]:
df_OWID_Covid = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
df_OWID_Testing = pd.read_csv('https://covid.ourworldindata.org/data/testing/covid-testing-latest-data-source-details.csv')
df_OWID_Countries = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/locations.csv')

### b. Read data from local datasets

Data downloaded manually from different databases, [European statistical database](https://ec.europa.eu/eurostat/data/database), [Wikipedia table on intensive care units](https://en.wikipedia.org/wiki/List_of_countries_by_hospital_beds) and [United Nations database](https://data.un.org):

In [1438]:
df_ESTAT_census = pd.read_csv('inputData/Eurostat_census_2001.csv')
df_WIKI_ICU = pd.read_csv('inputData/Wikipedia_ICU.csv')
df_UN_births = pd.read_csv('inputData/UNdata_birthsByMonth.csv')
df_UN_deaths = pd.read_csv('inputData/UNdata_deathsByMonth.csv')

### c. Query additional information for countries via wikipedia API

Additional Information
- Leader Gender
- Ideology of Leading Party
- Amount of Intensive Care Beds
- Gross domestic product per capita
- Human Development Index

In [1439]:
# Query for every tweet id in enhanced twitter archive and save tweet-information in json-format to 'tweet_json.txt'
'''             
country_jsons = {}
county_id_errors = []
start = time.time()
count = 0


with open('country_json.txt', 'w') as outfile:
    
    for country in df_JHU_Countries['Country/Region']:
        count +=1
        try:
            # Query API for data of wikipedia article
            article = wptools.page(country).get_parse()
            infobox = article.data['infobox']
            # Measure elapsed time
            mid_s = time.time()
            # Print id and time elapsed
            print(str(count) + str(mid_s - start) )
            # Write json of tweet to 'tweet_json.txt'
            json.dump(infobox, outfile)
            # New line
            outfile.write("\n")

        # Not best practice to catch all exceptions but fine for this short script
        except Exception as error:
            mid_f = time.time()
            print(str(count) + str(mid_f - start) + str(error))
            # Gather ids of id's without status
            tweet_id_errors.append([count, str(tweet_id)])
            
    end = time.time()
    print(end - start)
    
    '''

'             \ncountry_jsons = {}\ncounty_id_errors = []\nstart = time.time()\ncount = 0\n\n\nwith open(\'country_json.txt\', \'w\') as outfile:\n    \n    for country in df_JHU_Countries[\'Country/Region\']:\n        count +=1\n        try:\n            # Query API for data of wikipedia article\n            article = wptools.page(country).get_parse()\n            infobox = article.data[\'infobox\']\n            # Measure elapsed time\n            mid_s = time.time()\n            # Print id and time elapsed\n            print(str(count) + str(mid_s - start) )\n            # Write json of tweet to \'tweet_json.txt\'\n            json.dump(infobox, outfile)\n            # New line\n            outfile.write("\n")\n\n        # Not best practice to catch all exceptions but fine for this short script\n        except Exception as error:\n            mid_f = time.time()\n            print(str(count) + str(mid_f - start) + str(error))\n            # Gather ids of id\'s without status\n       

In [1440]:
'''
so = wptools.page('Germany').get_parse()
infobox = so.data['infobox']
print(infobox)
'''

"\nso = wptools.page('Germany').get_parse()\ninfobox = so.data['infobox']\nprint(infobox)\n"

<a id='assess'></a>
## 3. Assess data

After gathering each of the above pieces of data, they are assessed visually and programmatically for quality and tidiness issues. Requirements to be met:

- Quality requirements:
    - Completeness: All necessary records in dataframes, no specific rows, columns or cells missing.
    - Validity: No records available, that do not conform schema.
    - Accuracy: No wrong data, that is valid.
    - Consistency: No data, that is valid and accurate, but referred to in multiple correct ways.
- Tidiniss requirements (as defined by Hadley Wickham):
    - each variable is a column
    - each observation is a row
    - each type of observational unit is a table.

### a. Visual assessment

In [1441]:
# Check layout of df_JHU_Countries vsiually
df_JHU_Countries.sample(n=5)

,Unnamed: 0,Country/Region,inhabitants,area
2,2,France,65227357,551695
18,18,Greece,10439436,131940
1,1,Germany,83792987,357386
3,3,Belgium,11579502,30510
8,8,United Kingdom,67803450,242495


In [1442]:
# Check layout of df_JHU_Fatality vsiually
df_JHU_Fatality.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
106,NaN,Finland,64.0000,26.0000,0,0,0,0,0,0,...,255,260,265,267,271,275,284,287,293,297
36,British Columbia,Canada,49.2827,-123.1207,0,0,0,0,0,0,...,126,127,129,129,130,132,132,135,140,141
162,NaN,Montenegro,42.5000,19.3000,0,0,0,0,0,0,...,8,8,8,9,9,9,9,9,9,9
126,NaN,Haiti,18.9712,-72.2852,0,0,0,0,0,0,...,12,12,12,15,16,16,18,20,20,20
116,NaN,France,46.2276,2.2137,0,0,0,0,0,0,...,25949,26192,26271,26341,26604,26951,27032,27381,27485,27483


In [1443]:
# Check layout of df_JHU_Confirmed vsiually
df_JHU_Confirmed.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
43,Prince Edward Island,Canada,46.5107,-63.4168,0,0,0,0,0,0,...,27,27,27,27,27,27,27,27,27,27
225,NaN,US,37.0902,-95.7129,1,1,2,2,5,5,...,1257023,1283929,1309550,1329260,1347881,1369376,1390406,1417774,1442824,1467820
259,NaN,South Sudan,6.8770,31.3070,0,0,0,0,0,0,...,74,120,120,120,156,194,203,203,236,236
223,NaN,United Kingdom,55.3781,-3.4360,0,0,0,0,0,0,...,206715,211364,215260,219183,223060,226463,229705,233151,236711,240161
54,Guangdong,China,23.3417,113.4244,26,32,53,78,111,151,...,1589,1589,1589,1589,1589,1589,1589,1589,1589,1590


In [1444]:
# Check layout of df_JHU_Recovered vsiually
df_JHU_Recovered.sample(n=5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
68,Tianjin,China,39.3054,117.3230,0,0,0,0,0,0,...,186,186,186,186,186,186,187,187,187,187
127,NaN,Iran,32.0000,53.0000,0,0,0,0,0,0,...,82744,83837,85064,86143,87422,88357,89428,90539,91836,93147
15,Western Australia,Australia,-31.9505,115.8605,0,0,0,0,0,0,...,534,534,536,536,536,537,538,538,538,541
246,NaN,South Sudan,6.8770,31.3070,0,0,0,0,0,0,...,0,2,2,2,2,2,2,3,4,4
165,Sint Maarten,Netherlands,18.0425,-63.0548,0,0,0,0,0,0,...,44,44,46,46,46,46,46,46,46,54


In [1445]:
# Check layout of df_OWID_Covid vsiually
df_OWID_Covid.sample(n=5)

# df_OWID_Covidchange 'location' to 'country'
# df_OWID_Covid create df_OWID_Countries with 'iso_code', 'location', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_100k'
# df_OWID_Covid merge it to df_country

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
17786,NaN,International,2020-01-05,0,0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1404,BEL,Belgium,2020-05-03,49517,485,7765,62,4272.532,41.848,669.996,...,18.571,12.849,42658.576,0.2,114.898,4.29,25.1,31.4,NaN,5.640
4792,ECU,Ecuador,2020-01-24,0,0,0,0,0.000,0.000,0.000,...,7.104,4.458,10581.936,3.6,140.448,5.55,2.0,12.3,80.635,1.500
14175,ROU,Romania,2020-04-25,10417,321,552,25,541.489,16.686,28.694,...,17.850,11.690,23313.199,5.7,370.946,9.74,22.9,37.1,NaN,6.892
3530,COL,Colombia,2020-03-28,539,48,6,0,10.593,0.943,0.118,...,7.646,4.312,13254.949,4.5,124.240,7.44,4.7,13.5,65.386,1.710


In [1446]:
# Check layout of df_OWID_Testing vsiually
df_OWID_Testing.sample(n=5)

# df_OWID_Testing drop columns 'source URL', 'Source label', 'Notes', 'Number of observations', 'Daily change in cumulative total', 'Daily change in cumulative total per thousand', '3-day rolling mean daily change', '3-day rolling mean daily change per thousand', '7-day rolling mean daily change', '7-day rolling mean daily change per thousand','General source label', 'General source URL', 'Short description', 'Detailed description'
# df_OWID_Testing Either cut per regex country name from 'Entity' and rename country or join country name from other df

,ISO code,Entity,Date,Source URL,Source label,Notes,Number of observations,Cumulative total,Cumulative total per thousand,Daily change in cumulative total,Daily change in cumulative total per thousand,3-day rolling mean daily change,3-day rolling mean daily change per thousand,7-day rolling mean daily change,7-day rolling mean daily change per thousand,General source label,General source URL,Short description,Detailed description
12,COL,Colombia - samples tested,2020-05-15,https://www.ins.gov.co/Noticias/Paginas/Corona...,National Institute of Health,NaN,72,183112,3.599,6062.0,0.119,5791.333,0.114,6196.143,0.122,National Institute of Health,https://www.ins.gov.co/Noticias/Paginas/Corona...,The number of samples processed.,The Colombian National Institute of Health pub...
8,BRA,Brazil - tests performed,2020-04-20,https://www.saude.gov.br/noticias/agencia-saud...,Ministry of Health press release,NaN,2,132467,0.623,NaN,NaN,NaN,NaN,NaN,NaN,Brazil Ministry of Health,https://www.saude.gov.br/noticias/agencia-saude,The number of tests performed.,The Ministry of Health press releases publishe...
11,CHL,Chile - tests performed,2020-05-16,https://github.com/jorgeperezrojas/covid19-data,Ministry of Health,Made available by Jorge Perez Rojas on Github,46,350325,18.326,8813.0,0.461,12191.667,0.638,11774.429,0.616,Government of Chile Coronavirus information page,https://www.gob.cl/coronavirus/cifrasoficiales...,The number of tests performed,The Government of Chile release daily reports ...
81,TUN,Tunisia - units unclear,2020-05-14,https://github.com/kik00/TnCovid-19/blob/maste...,Tunisian Ministry of Health,NaN,65,37862,3.204,1339.0,0.113,1327.333,0.112,1491.714,0.126,Tunisian Ministry of Health,https://covid-19.tn/fr/tableau-de-bord/,Figures are provided both in terms of the numb...,The Tunisian Ministry of Health dashboard prov...
76,SWE,Sweden - people tested,2020-05-10,https://www.folkhalsomyndigheten.se/smittskydd...,Public Health Agency,NaN,11,177200,17.546,NaN,NaN,NaN,NaN,NaN,NaN,Public Health Agency,https://www.folkhalsomyndigheten.se/smittskydd...,The number of people tested.,The weekly report gives the cumulative total o...


In [1447]:
# Check layout of df_OWID_Countries vsiually
df_OWID_Countries.sample(n=5)

# df_OWID_Countries convert datatype population to integer
# df_OWID_Countries drop 'countriesAndTerritories', 'population_year'

,countriesAndTerritories,location,continent,population_year,population
0,Afghanistan,Afghanistan,Asia,2020.0,38928341.0
135,Namibia,Namibia,Africa,2020.0,2540916.0
29,Brunei_Darussalam,Brunei,Asia,2020.0,437483.0
63,Estonia,Estonia,Europe,2020.0,1326539.0
94,Iran,Iran,Asia,2020.0,83992953.0


In [1448]:
# Check layout of df_ESTAT_census vsiually
df_ESTAT_census.sample(n=5)

# df_ESTAT_census make columns from values in n_person
# df_ESTAT_census replace 'Germany (until 1990 former territory of the FRG)' with 'Germany'
# df_ESTAT_census drop country 'Bulgaria'
# df_ESTAT_census drop '4 persons', '5 persons', '6 persons or more'

,N_PERSON,GEO,TIME,AGE,HHCOMP,UNIT,Value,Flag and Footnotes
68,1 person,Hungary,2001,65 years or over,Total,Number,"905,836",NaN
51,No persons,Liechtenstein,2001,65 years or over,Total,Number,"10,796",NaN
46,No persons,Romania,2001,65 years or over,Total,Number,"4,967,443",NaN
72,1 person,Portugal,2001,65 years or over,Total,Number,"742,228",NaN
115,3 persons,Spain,2001,65 years or over,Total,Number,"70,716",NaN


In [1449]:
df_ESTAT_census.AGE.value_counts()

65 years or over    243
Name: AGE, dtype: int64

In [1450]:
# Check layout of df_WIKI_ICU vsiually
df_WIKI_ICU.sample(n=5)

,countryname,continent,hospital_beds_per_1000_people,occupancy,ICU-CCB_beds_per_1000_people,ventilators
9,France,Europe,5.98,75.6,11.6,"7,007"
0,Japan,Asia,13.05,75.5,7.3,"32,586"
38,Colombia,South America,1.70,NaN,NaN,NaN
8,Lithuania,Europe,6.56,73.2,15.5,"1,000"
12,Latvia,Europe,5.57,71.1,9.7,NaN


In [1451]:
df_UN_births.sample(n=5)
# Drop columns 'Area', 'Record Type', 'Reliability', 'Value Footnotes', 'Source Year'
# change datatype of columns  'Value' to integer
# Merge df_UN_births and df_UN_deaths on Year


,Country or Area,Year,Area,Month,Record Type,Reliability,Source Year,Value,Value Footnotes
5076,Kuwait,2016,Total,May,Data tabulated by year of occurrence,"Final figure, complete",2018.0,5055.0,NaN
6219,Mauritius,2010,Total,April,Data tabulated by year of occurrence,"Final figure, complete",2012.0,1272.0,27
1903,"China, Hong Kong SAR",2017,Total,January,Data tabulated by year of occurrence,"Final figure, complete",2018.0,5030.0,NaN
8413,Saint Helena ex. dep.,2012,Total,February,Data tabulated by year of occurrence,"Final figure, complete",2013.0,5.0,NaN
28,Åland Islands,2015,Total,February,Data tabulated by year of occurrence,"Final figure, complete",2017.0,24.0,NaN


In [1452]:
df_UN_births.Area.value_counts()

Total    10373
Name: Area, dtype: int64

In [1453]:
df_UN_deaths.sample(n=5)

,Country or Area,Year,Area,Month,Record Type,Reliability,Source Year,Value,Value Footnotes
7491,Republic of Moldova,2016,Total,April,Data tabulated by year of occurrence,"Final figure, complete",2018.0,3044.0,32
470,Aruba,2011,Total,Unknown,Data tabulated by year of occurrence,"Final figure, complete",2014.0,0.0,NaN
2818,Egypt,2011,Total,April,Data tabulated by year of occurrence,"Final figure, complete",2013.0,43083.0,NaN
7254,Qatar,2016,Total,March,Data tabulated by year of occurrence,"Final figure, complete",2018.0,201.0,NaN
2826,Egypt,2011,Total,December,Data tabulated by year of occurrence,"Final figure, complete",2013.0,39175.0,NaN


### b. Programmatic assessment

In [1454]:
# List of countries that are avaoilable in John Hopkins Dataset
df_JHU_Recovered['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Diamond Princess',
       'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia',
       'Germany', 'Ghana', 'Grenada', 'Greece', 'Guatemala', 'Guinea',
       'Guinea-Bissau', 'Guyana', 'Haiti', 'H

In [1455]:
# List of countries that are avaoilable in John Hopkins Dataset
df_OWID_Covid['location'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Armenia',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Bonaire Sint Eustatius and Saba',
       'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote d'Ivoire", 'Cameroon',
       'Democratic Republic of Congo', 'Congo', 'Colombia', 'Comoros',
       'Cape Verde', 'Costa Rica', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Dominica',
       'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt',
       'Eritrea', 'Western Sahara', 'Spain', 'Estonia', 'Ethiopia',
       'Finland', 'Fiji', 'Falkland Islands', 'France',

In [1456]:
# Available variables in dataset
list(df_OWID_Covid)

['iso_code',
 'location',
 'date',
 'total_cases',
 'new_cases',
 'total_deaths',
 'new_deaths',
 'total_cases_per_million',
 'new_cases_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'total_tests',
 'new_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'tests_units',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cvd_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_beds_per_100k']

In [1457]:
df_OWID_Covid.query('location == "Germany" and date == "2020-05-13"')

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
4244,DEU,Germany,2020-05-13,171306,798,7634,101,2044.616,9.524,91.115,...,21.453,15.957,45229.245,NaN,156.139,8.31,28.2,33.1,NaN,8.0


In [1458]:
df_Check = df_JHU_Confirmed.copy()
df_Check.rename(columns={'Country/Region': 'country'}, inplace=True)
df_Check.query('country == "Germany"')

,Province/State,country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
120,NaN,Germany,51.0,9.0,0,0,0,0,0,1,...,169430,170588,171324,171879,172576,173171,174098,174478,175233,175752


In [1459]:
# Check for countries which are referred to by different names in different dataframes
c_df_JHU_Fatality =  df_JHU_Fatality['Country/Region'].unique()
c_df_JHU_Confirmed = df_JHU_Confirmed['Country/Region'].unique()
c_df_JHU_Recovered = df_JHU_Recovered['Country/Region'].unique()
c_df_JHU_Countries = df_JHU_Countries['Country/Region'].unique()
c_df_OWID_Covid = df_OWID_Covid['location'].unique()
c_df_OWID_Testing = df_OWID_Testing['Entity'].unique()
c_df_OWID_Countries = df_OWID_Countries['location'].unique()
c_df_WIKI_ICU = df_WIKI_ICU['countryname'].unique()
c_df_UN_births = df_UN_births['Country or Area'].unique()
c_df_UN_deaths = df_UN_deaths['Country or Area'].unique()

all_country_names = list(c_df_JHU_Fatality) + list(c_df_JHU_Confirmed) + list(c_df_JHU_Recovered) + list(c_df_JHU_Countries) + list(c_df_OWID_Covid) + list(c_df_OWID_Testing) + list(c_df_OWID_Countries) + list(c_df_WIKI_ICU) + list(c_df_UN_births) + list(c_df_UN_deaths)

all_country_names = pd.Series(all_country_names).unique()

In [1460]:
all_country_names

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

### Findings, which contradict requirements:

#### Quality Observations:
- Validity: Some observations/rows in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' contain the values for a region, for example Australia appears multiple times in column country as the observations are per region.
- Consistency: Data about Covid-19 cases differs slightly between John Hopkins and OWID, data which is available in both datasets will be kept only from John Hopkins.
- Consistency: Some countries are referred to with varying names, for example 'US' and 'United Stats'. Other names are not valid.

#### Tidiness Observations:
- The data of 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' should be one observational unit 'df_covid' with columns 'country', 'date', 'recovered', 'confirmed', 'fatal' and 'date' beeing of type datetime.
- Column 'Country/Region' should only contain countries, therefore column name should by 'country', same for OWID data.
- Columns 'Province/State', 'Lat' and 'Long' are not necessary in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality'
- Data for countries, which are not of interested is not needed in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality', 'df_JHU_Countries'
- In the df_OWID_Covid dataframes there is covid-related data where the variation frequency is daily and there is data not directly covid-related where data variation frequency is monthly or even constant for . Thus, there should be three observational units, df_covid for covid-related data and daily observations, df_OWID_country.
- In the df_OWID_Testing dataframes there is data which is not of interest.






- df_ESTAT_census make columns from values in n_person
- df_ESTAT_census replace 'Germany (until 1990 former territory of the FRG)' with 'Germany'
- df_ESTAT_census drop country 'Bulgaria'
- df_ESTAT_census drop '4 persons', '5 persons', '6 persons or more'

-  merge df_OWID_Countries with df_country

<a id='clean'></a>
## 4. Clean data

In [1461]:
# Create copies for cleaning process to preserve original dataframes
df_JHU_Fatality_clean = df_JHU_Fatality.copy()
df_JHU_Confirmed_clean = df_JHU_Confirmed.copy()
df_JHU_Recovered_clean = df_JHU_Recovered.copy()
df_JHU_Countries_clean = df_JHU_Countries.copy()
df_OWID_Covid_clean = df_OWID_Covid.copy()
df_OWID_Testing_clean = df_OWID_Testing.copy()
df_OWID_Countries_clean = df_OWID_Countries.copy()
df_ESTAT_census_clean = df_ESTAT_census.copy()
df_WIKI_ICU_clean = df_WIKI_ICU.copy()
df_UN_births_clean = df_UN_births.copy()
df_UN_deaths_clean = df_UN_deaths.copy()

### Issue 1:
#### Observe:
-  Tidiness: Columns 'Province/State', 'Lat' and 'Long' are not necessary in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality'

#### Define:
- Drop columns 'Province/State', 'Lat' and 'Long'

#### Code:

In [1462]:
# Drop variables which are only necessary for retweets
df_JHU_Fatality_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)
df_JHU_Confirmed_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)
df_JHU_Recovered_clean.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)

#### Test:

In [1463]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Fatality_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,106,109,115,120,122,127,132,136,153,168


In [1464]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Confirmed_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,3563,3778,4033,4402,4687,4963,5226,5639,6053,6402


In [1465]:
# Check if columnns 'Province/State', 'Lat' and 'Long' dropped
df_JHU_Recovered_clean.head(1)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,468,472,502,558,558,610,648,691,745,745


### Issue 2:
#### Observe:
- Tidiness: Column 'Country/Region' should only contain countries, therefore column name should by 'Country'.

#### Define:
- Rename column 'Country/Region' to 'country'

#### Code:

In [1466]:
# Rename coloumn inplace to identic primary key
df_JHU_Fatality_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Confirmed_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Recovered_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_JHU_Countries_clean.rename(columns={'Country/Region': 'country'}, inplace=True)
df_OWID_Covid_clean.rename(columns={'location': 'country'}, inplace=True)
df_OWID_Countries_clean.rename(columns={'location': 'country'}, inplace=True)

#### Test:

In [1467]:
assert df_JHU_Fatality_clean.country.any()

In [1468]:
assert df_JHU_Confirmed_clean.country.any()

In [1469]:
assert df_JHU_Recovered_clean.country.any()

In [1470]:
assert df_JHU_Countries_clean.country.any()

In [1471]:
assert df_OWID_Covid_clean.country.any()

In [1472]:
assert df_OWID_Countries_clean.country.any()

### Issue 3:
#### Observe:
- Tidiness: In the df_OWID_Covid_clean dataframes there is covid-related data where the variation frequency is daily and there is data not directly covid-related where data variation frequency is monthly or even constant for . Thus, there should be three observational units, df_covid for covid-related data and daily observations, df_OWID_country.

#### Define
- Create new dataframe df_OWID_Countries with columns 'iso_code', 'location', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_100k'.

#### Code:

In [1473]:
df_OWID_Countries = df_OWID_Covid_clean.copy()
df_OWID_Countries.drop([ 'date',
                         'total_cases',
                         'new_cases',
                         'total_deaths',
                         'new_deaths',
                         'total_cases_per_million',
                         'new_cases_per_million',
                         'total_deaths_per_million',
                         'new_deaths_per_million',
                         'total_tests',
                         'new_tests',
                         'total_tests_per_thousand',
                         'new_tests_per_thousand',
                         'tests_units',
                         'cvd_death_rate',
                         'handwashing_facilities',
                         'extreme_poverty'], axis=1, inplace=True)
df_OWID_Countries = df_OWID_Countries.drop_duplicates()


#### Test:

In [1474]:
df_OWID_Countries.country.value_counts().head(3)

Mexico    1
Iran      1
Guinea    1
Name: country, dtype: int64

In [1475]:
list(df_OWID_Countries)

['iso_code',
 'country',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'hospital_beds_per_100k']

### Issue 4:
#### Observe:
- Validity: Some observations/rows in dataframes 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' contain the values for a region, for example Australia appears multiple times in column country as the observations are per region.

#### Define: 
- Sum values of rows with same entry in column country by using groupby

#### Code:

In [1476]:
# Groupby and sum
df_JHU_Fatality_clean = df_JHU_Fatality_clean.groupby(['country'], as_index=False).sum()
df_JHU_Confirmed_clean = df_JHU_Confirmed_clean.groupby(['country'], as_index=False).sum()
df_JHU_Recovered_clean = df_JHU_Recovered_clean.groupby(['country'], as_index=False).sum()

#### Test:

In [1477]:
df_JHU_Fatality_clean.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
183    False
184    False
185    False
186    False
187    False
Length: 188, dtype: bool

### Issue 5:
#### Observe:
- Tidiness: The data of 'df_JHU_Confirmed', 'df_JHU_Recovered', 'df_JHU_Fatality' should be one observational unit 'df_covid' with columns 'country', 'date', 'recovered', 'confirmed', 'fatal' and 'date' beeing of type datetime.

#### Define:
- Melt date columns to one column 'date', transform date to type datetime and merge the three dataframes to ones dataframe 'df_covid' with sorted date values.

#### Code:

In [1478]:
# Melt each dataframe so that results in columns: country,
df_JHU_Fatality_clean = pd.melt(df_JHU_Fatality_clean, id_vars = ['country'], var_name='date', value_name='fatal')
df_JHU_Confirmed_clean = pd.melt(df_JHU_Confirmed_clean, id_vars = ['country'], var_name='date', value_name='confirmed')
df_JHU_Recovered_clean = pd.melt(df_JHU_Recovered_clean, id_vars = ['country'], var_name='date', value_name='recovered')

In [1479]:
# Convert new columns date to datetime
df_JHU_Fatality_clean.date=pd.to_datetime(df_JHU_Fatality_clean.date)
df_JHU_Confirmed_clean.date=pd.to_datetime(df_JHU_Confirmed_clean.date)
df_JHU_Recovered_clean.date=pd.to_datetime(df_JHU_Recovered_clean.date)

In [1480]:
# Merge three covid dataframes to one
df_covid = pd.merge(df_JHU_Fatality_clean, df_JHU_Confirmed_clean, on=['country','date'])
df_covid = pd.merge(df_covid, df_JHU_Recovered_clean, on=['country','date'])

In [1481]:
# Sort date values by date
df_covid = df_covid.sort_values(by='date', ascending=True)

#### Test:

In [1482]:
list(df_covid)

['country', 'date', 'fatal', 'confirmed', 'recovered']

In [1483]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21808 entries, 0 to 21807
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   country    21808 non-null  object        
 1   date       21808 non-null  datetime64[ns]
 2   fatal      21808 non-null  int64         
 3   confirmed  21808 non-null  int64         
 4   recovered  21808 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1022.2+ KB


In [1484]:
df_covid

,country,date,fatal,confirmed,recovered
0,Afghanistan,2020-01-22,0,0,0
120,Namibia,2020-01-22,0,0,0
121,Nepal,2020-01-22,0,0,0
122,Netherlands,2020-01-22,0,0,0
123,New Zealand,2020-01-22,0,0,0
...,...,...,...,...,...
21684,Gambia,2020-05-16,1,23,12
21685,Georgia,2020-05-16,12,683,419
21686,Germany,2020-05-16,7938,175752,152600
21677,Estonia,2020-05-16,63,1770,934


### Issue 6:
#### Observe:
- Consistency: Some countries are referred to with varying names, for example 'US' and 'United Stats'. Other names are not valid.

#### Define:
- Clean names of all dataframes by extracting country names with regex expressions and renaming countries.

#### Code

In [1485]:
# Rename coloumn inplace
df_covid['country'].replace({'US': 'United States', 'Taiwan*': 'Taiwan'}, inplace=True)
df_JHU_Countries_clean['country'].replace({'US': 'United States', 'Taiwan*': 'Taiwan'}, inplace=True)

In [1486]:
# Extract country name from column 'Entity' via regex
df_OWID_Testing_clean['country'] = df_OWID_Testing_clean.Entity.str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

# Drop column from which country name was extracted
df_OWID_Testing_clean.drop(['Entity'], axis=1, inplace=True)

In [1487]:
# Extract country name from column 'Country or Area' via regex
df_UN_births_clean['country'] = df_UN_births_clean['Country or Area'].str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

df_UN_deaths_clean['country'] = df_UN_deaths_clean['Country or Area'].str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

# Drop column from which country name was extracted
df_UN_births_clean.drop(['Country or Area'], axis=1, inplace=True)
df_UN_deaths_clean.drop(['Country or Area'], axis=1, inplace=True)

In [1488]:
# Extract country name from column 'Entity' via regex
df_WIKI_ICU_clean['country'] = df_WIKI_ICU_clean.countryname.str.extract(
    '([A-Z][a-z]{0,20}( [A-Z][a-z]{0,20})?)', expand=True)[0]

# Drop column from which country name was extracted
df_WIKI_ICU_clean.drop(['countryname'], axis=1, inplace=True)

In [1489]:
# Rename coloumn inplace where different names refer to same country
df_UN_births_clean['country'].replace({'Russian Federation': 'Russia', 
                                       'British Virgin': 'British Virgin Islands',
                                       'Bosnia': 'Bosnia and Herzegovina',
                                       'Trinidad': 'Trinidad and Tobago',
                                       'Turks': 'Turks and Caicos Islands'}, inplace=True)

df_UN_deaths_clean['country'].replace({'Russian Federation': 'Russia', 
                                       'British Virgin': 'British Virgin Islands', 
                                       'Bosnia': 'Bosnia and Herzegovina',
                                       'Trinidad': 'Trinidad and Tobago',
                                       'Turks': 'Turks and Caicos Islands'}, inplace=True)


df_OWID_Countries_clean['country'].replace({'Macedonia': 'North Macedonia'}, inplace=True)


In [1490]:
# Create array with countries of interest
countries =['Afghanistan',
             'Albania',
             'Algeria',
             'American Samoa',
             'Andorra',
             'Angola',
             'Anguilla',
             'Antigua and Barbuda',
             'Argentina',
             'Armenia',
             'Aruba',
             'Australia',
             'Austria',
             'Azerbaijan',
             'Bahamas',
             'Bahrain',
             'Bangladesh',
             'Barbados',
             'Belarus',
             'Belgium',
             'Belize',
             'Benin',
             'Bermuda',
             'Bhutan',
             'Bolivia',
             'Bonaire Sint Eustatius and Saba',
             'Bosnia and Herzegovina',
             'Botswana',
             'Brazil',
             'British Virgin Islands',
             'Brunei',
             'Bulgaria',
             'Burkina Faso',
             'Burma',
             'Burundi',
             'Cabo Verde',
             'Cambodia',
             'Cameroon',
             'Canada',
             'Cape Verde',
             'Cayman Islands',
             'Central African Republic',
             'Chad',
             'Chile',
             'China',
             'Colombia',
             'Comoros',
             'Congo',
             'Cook Islands',
             'Costa Rica',
             "Cote d'Ivoire",
             'Croatia',
             'Cuba',
             'Cura',
             'Curacao',
             'Cyprus',
             'Czech Republic',
             'Czechia',
             'Denmark',
             'Diamond Princess',
             'Djibouti',
             'Dominica',
             'Dominican Republic',
             'Ecuador',
             'Egypt',
             'El Salvador',
             'Equatorial Guinea',
             'Eritrea',
             'Estonia',
             'Eswatini',
             'Ethiopia',
             'Faeroe Islands',
             'Falkland Islands',
             'Faroe Islands',
             'Fiji',
             'Finland',
             'France',
             'French Polynesia',
             'Gabon',
             'Gambia',
             'Georgia',
             'Germany',
             'Ghana',
             'Gibraltar',
             'Greece',
             'Greenland',
             'Grenada',
             'Guam',
             'Guatemala',
             'Guernsey',
             'Guinea',
             'Guinea-Bissau',
             'Guyana',
             'Haiti',
             'Holy See',
             'Honduras',
             'Hong Kong',
             'Hungary',
             'Iceland',
             'India',
             'Indonesia',
             'International',
             'Iran',
             'Iraq',
             'Ireland',
             'Islands',
             'Isle',
             'Isle of Man',
             'Israel',
             'Italy',
             'Jamaica',
             'Japan',
             'Jersey',
             'Jordan',
             'Kazakhstan',
             'Kenya',
             'Korea, South',
             'Kosovo',
             'Kuwait',
             'Kyrgyzstan',
             'Laos',
             'Latvia',
             'Lebanon',
             'Lesotho',
             'Liberia',
             'Libya',
             'Liechtenstein',
             'Lithuania',
             'Luxembourg',
             'MS Zaandam',
             'Madagascar',
             'Malawi',
             'Malaysia',
             'Maldives',
             'Mali',
             'Malta',
             'Mauritania',
             'Mauritius',
             'Mexico',
             'Moldova',
             'Monaco',
             'Mongolia',
             'Montenegro',
             'Montserrat',
             'Morocco',
             'Mozambique',
             'Myanmar',
             'Namibia',
             'Nepal',
             'Netherlands',
             'New Caledonia',
             'New Zealand',
             'Nicaragua',
             'Niger',
             'Nigeria',
             'North Macedonia',
             'Northern Mariana Islands',
             'Norway',
             'Oman',
             'Pakistan',
             'Palau',
             'Palestine',
             'Panama',
             'Papua New Guinea',
             'Paraguay',
             'Peru',
             'Philippines',
             'Poland',
             'Portugal',
             'Puerto Rico',
             'Qatar',
             'Republic',
             'Romania',
             'Russia',
             'Rwanda',
             'Saint Helena',
             'Saint Kitts and Nevis',
             'Saint Lucia',
             'Saint Vincent',
             'Saint Vincent and the Grenadines',
             'San Marino',
             'Sao Tome and Principe',
             'Saudi Arabia',
             'Senegal',
             'Serbia',
             'Seychelles',
             'Sierra Leone',
             'Singapore',
             'Sint Maarten (Dutch part)',
             'Slovakia',
             'Slovenia',
             'Somalia',
             'South Africa',
             'South Korea',
             'South Sudan',
             'Spain',
             'Sri Lanka',
             'Sudan',
             'Suriname',
             'Swaziland',
             'Sweden',
             'Switzerland',
             'Syria',
             'Taiwan',
             'Tajikistan',
             'Tanzania',
             'Thailand',
             'Timor',
             'Timor-Leste',
             'Togo',
             'Trinidad and Tobago',
             'Tunisia',
             'Turkey',
             'Turks and Caicos Islands',
             'Uganda',
             'Ukraine',
             'United Arab Emirates',
             'United Kingdom',
             'United States',
             'United States Virgin Islands',
             'Uruguay',
             'Uzbekistan',
             'Vatican',
             'Venezuela',
             'Vietnam',
             'West Bank and Gaza',
             'Western Sahara',
             'World',
             'Yemen',
             'Zambia',
             'Zimbabwe']

In [1491]:
# Only keep Countries of interes
df_covid = df_covid[df_covid['country'].isin(countries)]
df_JHU_Countries_clean = df_JHU_Countries_clean[df_JHU_Countries_clean['country'].isin(countries)]
df_OWID_Covid_clean = df_OWID_Covid_clean[df_OWID_Covid_clean['country'].isin(countries)]
df_OWID_Testing_clean = df_OWID_Testing_clean[df_OWID_Testing_clean['country'].isin(countries)]
df_OWID_Countries_clean = df_OWID_Countries_clean[df_OWID_Countries_clean['country'].isin(countries)]
df_WIKI_ICU_clean = df_WIKI_ICU_clean[df_WIKI_ICU_clean['country'].isin(countries)]
df_UN_births_clean = df_UN_births_clean[df_UN_births_clean['country'].isin(countries)]
df_UN_deaths_clean = df_UN_deaths_clean[df_UN_deaths_clean['country'].isin(countries)]

#### Test:

In [1492]:
# Visually check countries of all dataframes

print(list(pd.Series(df_covid['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_JHU_Countries_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_OWID_Covid_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_OWID_Testing_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_OWID_Countries_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_WIKI_ICU_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_UN_births_clean['country'].unique()).sort_values()))
print('\n\n')

print(list(pd.Series(df_UN_deaths_clean['country'].unique()).sort_values()))
print('\n\n')



['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', '

In [1493]:
# Check for countries which are referred to by different names in different dataframes
c_df_covid = df_covid['country'].unique()
c_df_JHU_Countries_clean = df_JHU_Countries_clean['country'].unique()
c_df_OWID_Covid_clean = df_OWID_Covid_clean['country'].unique()
c_df_OWID_Testing_clean = df_OWID_Testing_clean['country'].unique()
c_df_OWID_Countries_clean = df_OWID_Countries_clean['country'].unique()
c_df_WIKI_ICU_clean = df_WIKI_ICU_clean['country'].unique()
c_df_UN_births_clean = df_UN_births_clean['country'].unique()
c_df_UN_deaths_clean = df_UN_deaths_clean['country'].unique()

all_country_names = list(c_df_covid) + list(c_df_JHU_Countries_clean) + list(c_df_OWID_Covid_clean) + list(c_df_OWID_Testing_clean) + list(c_df_OWID_Countries_clean) + list(c_df_WIKI_ICU_clean) + list(c_df_UN_births_clean) + list(c_df_UN_deaths_clean)

# countries = pd.Series(all_country_names).sort_values()

# Create alphabetically sorted list of all countries in all dataframes
list(pd.Series(pd.Series(all_country_names).unique()).sort_values())

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire Sint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cura',
 'Curacao',
 'Cyprus',
 'Czech Republic',
 'Czechia',
 'Denmark',
 'Diamond Princess',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'E

In [1494]:
assert len(pd.Series(all_country_names).unique()) == len(countries)

### Issue 7:
#### Observe:
- Tidiness: In the df_OWID_Testing dataframes there is data which is not of interest.

#### Define:
- Drop columns 'source URL', 'Source label', 'Notes', 'Number of observations', 'Daily change in cumulative total', 'Daily change in cumulative total per thousand', '3-day rolling mean daily change', '3-day rolling mean daily change per thousand', '7-day rolling mean daily change', '7-day rolling mean daily change per thousand','General source label', 'General source URL', 'Short description', 'Detailed description' from df_OWID_Testing_clean

#### Code

In [1499]:
# Drop columns
df_OWID_Testing_clean.drop(['7-day rolling mean daily change per thousand', 
                            '3-day rolling mean daily change per thousand', 
                            'Daily change in cumulative total per thousand', 
                            '3-day rolling mean daily change', 
                            '7-day rolling mean daily change', 'Number of observations', 
                            'Daily change in cumulative total', 'General source label',
                            'General source URL', 'Short description', 'Source URL', 'Source label', 'Notes',  
                            'Detailed description'], axis=1, inplace=True)

#### Test

In [1500]:
list(df_OWID_Testing_clean)

['ISO code',
 'Date',
 'Cumulative total',
 'Cumulative total per thousand',
 'country']

### Issue 8:
#### Observe:
- Datatype of variable population is not integer and there are variables not of interest for this project 'countriesAndTerritories' and 'population_year'

#### Define:
- Convert datatype of population to integer, drop columns 'countriesAndTerritories', 'population_year'



#### Code

#### Test

<a id='store'></a>
## 5. Store clean data

In [ ]:
# Store cleaned dataset to csv
df_covid.to_csv('covid_master.csv', encoding='utf-8')